In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, concatenate, Input, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras import backend as K
from tensorflow import keras

2023-07-12 10:27:43.504165: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-12 10:27:43.674445: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-12 10:27:43.675378: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-12 10:27:44.465906: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def load_data(csv_path, image_dir):
    # Load CSV data
    df = pd.read_csv(csv_path)
    X = np.array(df.drop(["diagnostic", "img_id"], axis=1))
    y = np.array(df["diagnostic"])
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
    y = to_categorical(y)

    # Load images using ImageDataGenerator
    datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    batch_size = 600
    target_size = (224, 224)
    class_mode = 'categorical'

    train_generator = datagen.flow_from_dataframe(
        dataframe=df,
        directory=image_dir,
        x_col="img_id",
        y_col="diagnostic",
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode,
        shuffle=True
    )

    xtrain, ytrain = next(train_generator)
    return xtrain, ytrain, X, y

In [3]:
def build_cnn_model(input_shape):
    BASE_MODEL = ResNet50(
        weights='imagenet',
        include_top=False,
        pooling='avg',
        input_shape=input_shape
    )
    BASE_MODEL.trainable = False

    model = Sequential()
    model.add(BASE_MODEL)
    model.add(Dense(units=2048, activation="elu"))
    model.add(Dense(units=512, activation="elu"))
    model.add(Dense(units=128, activation="elu"))
    model.add(Dropout(rate=0.5))
    return model

def build_densenet_model(input_shape):
    BASE_MODEL_DENSENET = DenseNet121(
        weights='imagenet',
        include_top=False,
        pooling='avg',
        input_shape=input_shape
    )
    BASE_MODEL_DENSENET.trainable = False

    model = Sequential()
    model.add(BASE_MODEL_DENSENET)
    model.add(Dense(units=512, activation="relu"))
    model.add(Dropout(rate=0.5))
    return model

def build_mobilenet_model(input_shape):
    BASE_MODEL_MOBILENET = MobileNet(
        weights='imagenet',
        include_top=False,
        pooling='avg',
        input_shape=input_shape
    )
    BASE_MODEL_MOBILENET.trainable = False

    model = Sequential()
    model.add(BASE_MODEL_MOBILENET)
    model.add(Dropout(rate=0.5))
    return model

def build_ann_model(input_shape):
    model = Sequential()
    input_ann = Input(shape=input_shape)
    ann_output = Dense(units=32, activation='sigmoid')(input_ann)
    ann_output = Dense(units=16, activation='sigmoid')(ann_output)
    ann_output = Dense(units=8, activation='sigmoid')(ann_output)
    ann_output = Dense(units=6, activation='sigmoid')(ann_output)
    model = Model(inputs=input_ann, outputs=ann_output)
    return model

def build_combined_model(cnn_model, ann_model):
    concatenated = concatenate([cnn_model.output, ann_model.output])
    combined_output = Dense(units=128, activation='relu')(concatenated)
    combined_output = Dense(units=64, activation='relu')(combined_output)
    combined_output = Dense(units=32, activation='relu')(combined_output)
    combined_output = Dense(units=16, activation='relu')(combined_output)
    combined_output = Dense(units=8, activation='relu')(combined_output)
    combined_output = Dense(units=6, activation='softmax')(combined_output)

    combined_model = Model(
        inputs=[cnn_model.input, ann_model.input],
        outputs=[combined_output]
    )
    return combined_model

def train_model(model, xtrain, ytrain, X, y, batch_size=20, epochs=1000):
    opt = tf.keras.optimizers.SGD(lr=0.001)
    model.compile(optimizer=opt, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy()])
    
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=55)
    
    history = model.fit([xtrain, X], ytrain, batch_size=batch_size, epochs=epochs,
                        callbacks=[stop_early], validation_split=1/9)
    
    return history


In [4]:
# Clear Keras session
K.clear_session()

# Load and preprocess the data
csv_path = "Augmentation/train_df.csv"
image_dir = "cancer/images/train"
xtrain, ytrain, X, y = load_data(csv_path, image_dir)

Found 600 validated image filenames belonging to 6 classes.


In [5]:
# Build individual models
cnn_model = build_cnn_model(input_shape=(224, 224, 3))
#densenet_model = build_densenet_model(input_shape=(224, 224, 3))
#mobilenet_model = build_mobilenet_model(input_shape=(224, 224, 3))
ann_model = build_ann_model(input_shape=X.shape[1])

2023-07-12 10:28:02.455889: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [6]:
# Build combined model
combined_model = build_combined_model(cnn_model, ann_model)
combined_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 resnet50_input (InputLayer)    [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 resnet50 (Functional)          (None, 2048)         23587712    ['resnet50_input[0][0]']         
                                                                                                  
 dense (Dense)                  (None, 512)          1049088     ['resnet50[0][0]']               
                                                                                                  
 input_2 (InputLayer)           [(None, 57)]         0           []                         

In [7]:
# Train the model
history = train_model(combined_model, xtrain, ytrain, X, y, batch_size=10, epochs=1000)

Epoch 1/1000
2/2 [==============================] - 32s 15s/step - loss: 1.8269 - categorical_accuracy: 0.1595 - val_loss: 1.8120 - val_categorical_accuracy: 0.2388
Epoch 2/1000


KeyboardInterrupt: 

In [ ]:

# Save the model
combined_model.save("model_moons_3.h5")

# Save the model as picle
'''
import pickle
from tensorflow.keras.models import load_model

model = load_model('model_moons_3.h5')

with open('model_moons_3.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('model_moons_3.pkl', 'rb') as f:
    model = pickle.load(f)
'''
